In [818]:
import logging
from pprint import pprint, pformat
from collections import namedtuple
import random
from copy import deepcopy

In [819]:
Nimply = namedtuple("Nimply", "row, num_objects")

In [820]:
class Nim:
    def __init__(self, num_rows: int, k: int = None) -> None:
        self._rows = [i * 2 + 1 for i in range(num_rows)]
        self._k = k

    def __bool__(self):
        return sum(self._rows) > 0

    def __str__(self):
        return "<" + " ".join(str(_) for _ in self._rows) + ">"

    @property
    def rows(self) -> tuple:
        return tuple(self._rows)

    def nimming(self, ply: Nimply) -> None:
        row, num_objects = ply
        assert self._rows[row] >= num_objects
        assert self._k is None or num_objects <= self._k
        self._rows[row] -= num_objects

class individual:
    def __init__(self, odd: float, first_last: float, k: float, wins: int) -> None:
        self._odd = odd
        self._first_last = first_last
        self._k =  k
        self._wins = wins

    def __str__(self):
        return f"the individual won {self.oddParam} matches"

    @property
    def oddParam(self) -> float:
        return self._odd

    @property
    def FirstLast(self) -> float:
        return self._first_last
    
    @property
    def k(self) -> float:
        return self._k
    
    @property
    def wins(self) -> int:
        return self._wins
    
    def addWin(self) -> None:
        self._wins += 1
    
    def winsNumber(self) -> str:
        return f""

In [821]:
def pure_random(state: Nim) -> Nimply:
    """A completely random move"""
    row = random.choice([r for r, c in enumerate(state.rows) if c > 0])
    num_objects = random.randint(1, state.rows[row])
    return Nimply(row, num_objects)

def gabriele(state: Nim) -> Nimply:
    """Pick always the maximum possible number of the lowest row"""
    possible_moves = [(r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1)]
    return Nimply(*max(possible_moves, key=lambda m: (-m[0], m[1])))

In [822]:
import numpy as np

def nim_sum(state: Nim) -> int:
    tmp = np.array([tuple(int(x) for x in f"{c:032b}") for c in state.rows])
    xor = tmp.sum(axis=0) % 2
    return int("".join(str(_) for _ in xor), base=2)

def analize(raw: Nim) -> dict:
    cooked = dict()
    cooked["possible_moves"] = dict()
    for ply in (Nimply(r, o) for r, c in enumerate(raw.rows) for o in range(1, c + 1)):
        tmp = deepcopy(raw)
        tmp.nimming(ply)
        cooked["possible_moves"][ply] = nim_sum(tmp)
    return cooked

def optimal(state: Nim) -> Nimply:
    analysis = analize(state)
    logging.debug(f"analysis:\n{pformat(analysis)}")
    spicy_moves = [ply for ply, ns in analysis["possible_moves"].items() if ns != 0]
    if not spicy_moves:
        spicy_moves = list(analysis["possible_moves"].keys())
    ply = random.choice(spicy_moves)
    return ply

In [823]:
def count_odd_rows(state: Nim):
    odd_index = []
    index = 0
    for r in state.rows:
        if r % 2:
            odd_index.append(index)
        index = index + 1
    return odd_index

def rows_with_element(state: Nim):
    rowsWithElement = []
    index = 0
    for r in state.rows:
        if r > 0:
            rowsWithElement.append(index)
        index = index + 1
    return rowsWithElement

def ES(state: Nim, I: individual) -> Nimply:
    """Evolutionary Strategy"""
    odd_index = count_odd_rows(state)
    odd_ratio = len(odd_index)/ len(state.rows)
    if (odd_ratio > I.oddParam) & (odd_ratio != 0.0):
        if random.random() < I.k: N = state.rows[odd_index[0]]
        else: N = 1
        return Nimply(odd_index[0], N)
    else:
        rowsWithElement = (rows_with_element(state))
        L = len(rowsWithElement)-1
        if random.random() < I.FirstLast:
            if random.random() < I.k: return Nimply(rowsWithElement[0], state.rows[rowsWithElement[0]])
            else: return Nimply(rowsWithElement[0], 1)
        else:
            if random.random() < I.k: return Nimply(rowsWithElement[L], state.rows[rowsWithElement[L]])
            else: return Nimply(rowsWithElement[L], 1)

def ES_match(ES_agent: individual, rival: None) -> int:
    win_count = 0
    for matches in range(1, 101):
        player = matches % 2
        nim = Nim(5)
        while nim:
            if player == 0: ply = ES(nim, ES_agent)
            else: ply = rival(nim)
            nim.nimming(ply)
            player = 1 - player
        if(player == 0):
            win_count = win_count + 1 
    return win_count

In [824]:
def mutation(off_spring: None) -> None:
    probability = 0.2
    for I in off_spring:
        I._wins = 0
        if random.random() < probability: I._odd = random.random()
        if random.random() < probability: I._k = random.random()
        if random.random() < probability: I._first_last = random.random()
    return off_spring

def new_individuals(off_spring: None) -> None:
    NEW = []
    for _ in range(50):
        parents = (off_spring[random.randint(0, 49)], off_spring[random.randint(0, 49)])
        new_I = individual(parents[random.randint(0,1)].oddParam, parents[random.randint(0,1)].FirstLast, parents[random.randint(0,1)].k, 0)
        NEW.append(new_I)
    return NEW

In [825]:
LAMB = 100
COUNT = 1
ROWS = 5

off_spring = np.array([individual(random.random(), random.random(), random.random(), 0) for _ in range(LAMB)])

for _ in range(10):    
    for I in off_spring:
        for matches in range(LAMB - COUNT):
            for first in (0,1):
                player = first
                nim = Nim(ROWS)
                agents = (I, off_spring[COUNT+matches])
                while nim:
                    ply = ES(nim, agents[player])
                    nim.nimming(ply)
                    player = 1 - player
                agents[player].addWin()
        COUNT += 1
    off_spring = sorted(off_spring, key=lambda x: x.wins, reverse=True)
    off_spring = off_spring[:50]
    off_spring = mutation(off_spring)
    for _ in range(50):
        parents = (off_spring[random.randint(0, 49)], off_spring[random.randint(0, 49)])
        new_I = individual(parents[random.randint(0,1)].oddParam, parents[random.randint(0,1)].FirstLast, parents[random.randint(0,1)].k, 0)
        off_spring.append(new_I)
    COUNT = 1

best = off_spring[0]

print(f"Evolutionary strategy won against random strategy {ES_match(best, pure_random)} times")
print(f"Evolutionary strategy won against optimal strategy {ES_match(best, optimal)} times")

Evolutionary strategy won against random strategy 67 times
Evolutionary strategy won against optimal strategy 37 times
